In [13]:
# Import Dependencies

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pylab import *
from config import password

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, MetaData

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float

import pymysql
pymysql.install_as_MySQLdb()

In [14]:
# Create references to the CSVs and import into Pandas DataFrames

measurements_csv = os.path.join("clean_hawaii_measurements.csv")
stations_csv = os.path.join("clean_hawaii_stations.csv")

measurements_df = pd.read_csv(measurements_csv, low_memory=False)
stations_df = pd.read_csv(stations_csv, low_memory=False)

In [15]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

# Create a connection to the engine called `conn`
conn = engine.connect()

In [16]:
# Set an object to use `declarative_base` from SQLAlchemy to model tables as ORM classes
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

# Pass this class to convert classes to sql tables
# Make sure to specify types for each column, e.g. Integer, Text, etc.

class Stations(Base):
    __tablename__ = 'stations'

    station = Column(String, primary_key=True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
class Measurements(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
# Use `create_all` to create tables in the database
Base.metadata.create_all(engine)

In [17]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options

data_stations = stations_df.to_dict(orient='records')

data_measurements = measurements_df.to_dict(orient='records')

In [18]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [19]:
# Save the reference to the `stations` table as a variable called `table`
# Needed to bulk upload data, rather than add through individual objects
table_stations = sqlalchemy.Table('stations', metadata, autoload=True)

table_measurements = sqlalchemy.Table('measurements', metadata, autoload=True)

In [20]:
# Open the session
# Session = sessionmaker(bind=engine)
# session = Session()

# Create our session (link) from Python to the DB
session = Session(engine)

In [21]:
# Use `table.delete()` to remove any pre-existing data.
conn.execute(table_stations.delete())
conn.execute(table_measurements.delete())

# Use `table.insert()` to insert the data into the table
conn.execute(table_stations.insert(), data_stations)
conn.execute(table_measurements.insert(), data_measurements)



In [22]:
# Commit the changes
session.commit()

# Close the session
session.close()

In [23]:
conn.execute("SELECT * FROM stations LIMIT 5").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [24]:
conn.execute("SELECT * FROM measurements LIMIT 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0490204841713224, 73)]